In [2]:
import numpy as np
import pandas as pd
from keras.layers import Dense, Input, LSTM, Bidirectional, Conv1D
from keras.layers import Dropout, Embedding
from keras.preprocessing import text, sequence
from keras.layers import GlobalMaxPooling1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D
from keras.models import Model
from keras.models import Sequential
import numpy as np
import pandas as pd
from keras.layers import Dense, Input, LSTM, Bidirectional, Conv1D
from keras.layers import Dropout, Embedding
from keras.preprocessing import text, sequence
from keras.layers import GlobalMaxPooling1D, GlobalAveragePooling1D, concatenate, SpatialDropout1D
from keras.models import Model

Using TensorFlow backend.
/Users/natepill/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.7
  return f(*args, **kwds)


In [39]:

EMBEDDING_FILE = 'data/glove.6B.300d.txt'
train_x = pd.read_csv('data/train.csv').fillna(" ")
test_x = pd.read_csv('data/test.csv').fillna(" ")

max_features=100000
maxlen=150
embed_size=300


In [40]:
train_x.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [29]:
len(train_x.loc[train_x["toxic"] == 1])

15294

In [30]:
len(train_x.loc[train_x["severe_toxic"] == 1])

1595

In [31]:
len(train_x.loc[train_x["obscene"] == 1])

8449

In [32]:
len(train_x.loc[train_x["threat"] == 1])

478

In [33]:
len(train_x.loc[train_x["insult"] == 1])

7877

In [34]:
len(train_x.loc[train_x["identity_hate"] == 1])

1405

In [41]:

train_x['comment_text'].fillna(' ')
test_x['comment_text'].fillna(' ')
train_y = train_x[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']].values
train_x = train_x['comment_text'].str.lower()

test_x = test_x['comment_text'].str.lower()



In [42]:
# Vectorize text + Prepare GloVe Embedding
tokenizer = text.Tokenizer(num_words=max_features, lower=True)
tokenizer.fit_on_texts(list(train_x))

train_x = tokenizer.texts_to_sequences(train_x)
test_x = tokenizer.texts_to_sequences(test_x)

train_x = sequence.pad_sequences(train_x, maxlen=maxlen)
test_x = sequence.pad_sequences(test_x, maxlen=maxlen)


In [43]:
embeddings_index = {}
with open(EMBEDDING_FILE, encoding='utf8') as f:
    for line in f:
        values = line.rstrip().rsplit(' ')
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

word_index = tokenizer.word_index
num_words = min(max_features, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, embed_size))
for word, i in word_index.items():
    if i >= max_features:
        continue
    
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

KeyboardInterrupt: 

In [ ]:
model = Sequential()
model.add(Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=True))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Conv1D(128, kernel_size=3, padding='valid', kernel_initializer='glorot_uniform'))
model.add(GlobalMaxPooling1D())

# Set to 2 output
model.add(Dense(6, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

In [ ]:

# Prediction
batch_size = 32
epochs = 1

model.fit(train_x, train_y, batch_size=batch_size, epochs=epochs, verbose=1)


In [47]:
testing = test_x[0:1000]

In [48]:
predictions = model.predict(testing, verbose=1)

1000/1000 [==============================] - 7s 7ms/step


### TODO:
* Upsample the minority class

* Eval Metrics:
Select One
Sensitivity(Recall) and specificity, Percision, F1 Score
Calculate both for training and test

AUC 

Focus on model eval for LSTM and then compare to CNN performance

Hyperparamter tuning for LSTM and CNN models

NOTE: there should be balance between sensitivity, specificity
